## Imports

In [1]:
import torch
from torchvision import transforms
from torchvision.transforms import Compose, Resize
from torchvision.datasets import CocoDetection
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
import fiftyone
import json
import cv2
import os
import shutil
from ultralytics import YOLO
import torch.optim as optim


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## Load data

In [11]:
categories = ["person","bicycle","car","motorcycle", "airplane","bus","train","truck","boat","traffic light","fire hydrant","stop sign","parking meter","bench","bird","cat","dog","horse"]

# Load Training dataset
dataset = fiftyone.zoo.load_zoo_dataset(
    "coco-2017",
    splits=["train",'validation'],
    label_types="detections",
    only_matching=True,
    classes=categories,
    dataset_dir = './dataset'
    )


 100% |██████|    1.9Gb/1.9Gb [16.5s elapsed, 0s remaining, 114.7Mb/s]      
Extracting annotations to '/Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset/raw/instances_train2017.json'
 100% |██████████████| 84853/84853 [1.0h elapsed, 0s remaining, 20.2 images/s]      
Writing annotations for 84853 downloaded samples to '/Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset/train/labels.json'
Found annotations at '/Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset/raw/instances_val2017.json'
 100% |████████████████| 3574/3574 [2.7m elapsed, 0s remaining, 20.1 images/s]      
Writing annotations for 3574 downloaded samples to '/Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset/validation/labels.json'
Dataset info written to '/Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset/info.json'
Loading 'coco-2017' split 'train'
 100% |█████████████| 84853/84853 [5.8m elapsed, 0s remaining, 241.2 samples/s]      
Loading 'coco-2017' sp

'\ndataset_train.export(\n    export_dir="/Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset/train",\n    dataset_type=fiftyone.types.COCODetectionDataset,\n    label_field="ground_truth"\n)\n'

## Bring Data into YOLO format

In [ ]:
# Define pathes
input_path = './dataset'

from ultralytics.data.converter import convert_coco

#train yolo format
convert_coco(labels_dir='/Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset/train',save_dir = './converter_result')
shutil.move('./converter_result/labels/labels', os.path.join(input_path,'train/labels'))
shutil.rmtree('./converter_result')
os.rename(os.path.join(input_path,'train/data'), os.path.join(input_path,'train/images'))


#validation yolo format
convert_coco(labels_dir='/Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset/validation',save_dir = './converter_result')
shutil.move('./converter_result/labels/labels', os.path.join(input_path,'validation/labels'))
shutil.rmtree('./converter_result')
os.rename(os.path.join(input_path,'validation/data'), os.path.join(input_path,'validation/images'))


In [52]:
# Utilities

# Read Json file
def read_json_file(path):
    f = open(path)
    data = json.load(f)
    f.close()
    return data

# Load Json anotation: dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])
train_labels = read_json_file(os.path.join(input_path,'train/labels.json'))
val_labels = read_json_file(os.path.join(input_path,'validation/labels.json'))

## Yolo Benchmark:



In [ ]:
from ultralytics import YOLO
model = YOLO('./model/yolov8n.pt') 
metrics = model.val(data='coco.yaml')

## YOLO Training

In [80]:
model = YOLO('./model/yolov8n.pt')  

# Train the model
results = model.train(data='coco.yaml', epochs=1, imgsz=640)
#results = model.train(data='coco.yaml', epochs=100, imgsz=640, device=[0, 1])

New https://pypi.org/project/ultralytics/8.1.29 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.27 🚀 Python-3.9.12 torch-2.2.1 CPU (Intel Core(TM) i9-9880H 2.30GHz)
engine/trainer: task=detect, mode=train, model=./model/yolov8n.pt, data=coco.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=F

train: Scanning /Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset/train/labels... 84853 images, 0 backgrounds, 0 corrupt: 100%|██████████| 84853/84853 [00:42<00:00, 2014.63it/s]


train: New cache created: /Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset/train/labels.cache


val: Scanning /Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset/validation/labels.cache... 3574 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3574/3574 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000119, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)


## YOLO Inference

In [76]:
from ultralytics import YOLO
model = YOLO('./model/yolov8n.pt') 
metrics = model.val(data='coco.yaml')

Ultralytics YOLOv8.1.27 🚀 Python-3.9.12 torch-2.2.1 CPU (Intel Core(TM) i9-9880H 2.30GHz)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


val: Scanning /Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset/validation/labels... 3574 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3574/3574 [00:01<00:00, 1869.00it/s]


val: New cache created: /Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset/validation/labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 224/224 [03:48<00:00,  1.02s/it]


                   all       3574      29168      0.607      0.421      0.462      0.324
                person       3574      10777      0.759      0.673      0.747      0.516
               bicycle       3574        314      0.687      0.392      0.458      0.265
                   car       3574       1918      0.649      0.516      0.563      0.364
            motorcycle       3574        367      0.717       0.58      0.659      0.415
              airplane       3574        143      0.833      0.766      0.835      0.654
                   bus       3574        283       0.74      0.643      0.739      0.621
                 train       3574        190      0.794      0.768      0.836      0.647
                 truck       3574        414      0.554      0.402      0.437      0.295
                  boat       3574        424       0.58        0.3      0.379      0.211
         traffic light       3574        634      0.655      0.347      0.414      0.213
          fire hydran

## OLD

In [12]:
# Utils functions



# Copy files from one folder to another
def copy_files_to_folder(source, destination):
    count = 0
    file_names = []
    for filename in os.listdir(source):
        source_file = os.path.join(source,filename)
        destination_file = f"{destination}/img{count}.jpg"

        try:
            shutil.copy(source_file, destination_file)
        except shutil.SameFileError:
            print("Source and destination represents the same file.")

        file_names.append(filename)
        count += 1

    return file_names

# Get image annotation
def get_img_ann(image_id,labels_dict):
    img_ann = []
    isFound = False
    for ann in labels_dict['annotations']:
        if ann['image_id'] == image_id:
            img_ann.append(ann)
            isFound = True
    if isFound:
        return img_ann
    else:
        return None
    

# Get image
def get_img(filename,labels_dict):
	for img in labels_dict['images']:
		if img['file_name'] == filename:
			return img


# Create Labels in YOLO format
          
def create_labels_yolo(file_names,labels_dict,output_path):

    count = 0

    for filename in file_names:
        # Extracting image 
        img = get_img(filename,labels_dict)
        img_id = img['id']
        img_w = img['width']
        img_h = img['height']

        # Get Annotations for this image
        img_ann = get_img_ann(img_id,labels_dict)

        if img_ann:
            # Opening file for current image
            file_object = open(f"{output_path}/labels/img{count}.txt", "a")

            for ann in img_ann:
                current_category = ann['category_id'] - 1 # As yolo format labels start from 0 
                current_bbox = ann['bbox']
                x = current_bbox[0]
                y = current_bbox[1]
                w = current_bbox[2]
                h = current_bbox[3]

                # Finding midpoints
                x_centre = (x + (x+w))/2
                y_centre = (y + (y+h))/2

                # Normalization
                x_centre = x_centre / img_w
                y_centre = y_centre / img_h
                w = w / img_w
                h = h / img_h

                # Limiting upto fix number of decimal places
                x_centre = format(x_centre, '.6f')
                y_centre = format(y_centre, '.6f')
                w = format(w, '.6f')
                h = format(h, '.6f')
                    
                # Writing current object 
                file_object.write(f"{current_category} {x_centre} {y_centre} {w} {h}\n")

            file_object.close()
            count += 1  # This should be outside the if img_ann block.


# Torch utilities:

def get_resize_size(transform):
    for t in transform.transforms:
        if isinstance(t, Resize):
            return t.size
    return None


In [ ]:
# Define pathes
input_path = '/Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset'
output_path = '/Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset_yolo'



# Copy Image files to the correct folder
train_image_names = copy_files_to_folder(os.path.join(input_path,'train/data'),os.path.join(output_path,'train/images'))
val_image_names = copy_files_to_folder(os.path.join(input_path,'validation/data'),os.path.join(output_path,'validation/images'))




In [20]:
# Create Labels in Yolo format
create_labels_yolo(train_image_names,train_labels,os.path.join(output_path,'train'))
create_labels_yolo(val_image_names,val_labels,os.path.join(output_path,'validation'))